# Solving 3-SAT Problem with Hybrid Quantum-Classical solver 

**Author:** Xinghui Jia \& Siwei Tan

**Date:** 9/4/2024

Based on paper "[Janus-CT: A Hybrid Approach for 3-SAT Problems by Integrating Quantum Annealer with CDCL][1]"

[1]: https://ieeexplore.ieee.org/abstract/document/10071022/


A propositional satisfiability (SAT) problem is to find an assignment for each variable to satisfy a given Boolean formula. A typical SAT problem is composed of multiple clauses represented as a disjunction of Boolean variables. 3-SAT problem is a special case of the SAT problem that has no more than three variables in each clause. 

3-SAT problem is a fundamental problem in various applications, such as artificial intelligence, circuit analysis, and protein structure prediction. Since the 3-SAT problem is an NP-complete problem, the time complexity of classical algorithms increases exponentially with the number of clauses and variables. We observe that the classical CDCL algorithm has the advantage of solving larger problems, while QA is more suitable for solving small but hard problems. To combine their advantages, we introduce Janus-SAT in this notebook, a hybrid approach that integrates quantum annealing (QA) with the classical Conflict-Driven Clause Learning (CDCL) algorithm to enable end-to-end acceleration for solving 3-SAT problems. 


<div style="text-align:center;">
    <img src="./pictures/4-1.hyqsat.png"  width="45%" height="45%">
</div>

The figure above shows the workflow of Janus-CT. It features a cross-iterative process. The CDCL algorithm searches for the solution. During the search, it identifies and sends the hard sub-problem to QA. Janus-CT designed a fast compilation flow to embed the problem and optimize the noise. It also designs multiple strategies to guide the search of CDCL using the QA results.

In [1]:
import sys
import os
import logging

sys.path.append('..')

from janusq.hyqsat import solve_by_janusct, solve_by_minisat

In [2]:
# Configure the model
file_path = "cnf_examples/UF100/uf100-01.cnf"  # input cnf flie
cpu_lim = 0  # limit the cpu time (s). 0 means infinite
mem_lim = 0  # limit the memory 0 means infinite
strictp = True  # isStrict
use_realQC = False # use real quantum computer provided by dwave, but you need to get dwave auth.

### Using Janus-SAT API to Solve the Problem

We can use a QA simulator or real-world QA hardware to solve the SAT roblem. For example, here we use the simualtor.

In [3]:
# solve by Janus-SAT
solve_by_janusct(file_path, verb=True, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 1,
 'conflicts': 8,
 'conflict cost': 0.069,
 'decisions': 0,
 'decisions cost': 0.03,
 'propagations': 360,
 'propagations cost': 0.325,
 'conflict literals': 59,
 'actual CPU time': 7.44417,
 'solving time': 1.1,
 'annealing time': 0.46,
 'simulation time': 1.25079,
 'quantum success number': 4,
 'quantum conflict number': 19,
 'quantum one time solve number': 0,
 'is satisfiable': True,
 'is sat': True}

Note that when using the simulator, the solving time is estimated as (CDCL time + number of QA $\times$ 120 $\mu s$).

### Comparision to MiniSAT

We compare it to the MiniSAT solver.

In [4]:
# solve by minisat
solve_by_minisat(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 3,
 'conflicts': 384,
 'conflict cost': 2.866,
 'decisions': 463,
 'decisions cost': 0.47,
 'propagations': 9130,
 'propagations cost': 5.591,
 'conflict literals': 2416,
 'actual CPU time': 0.097112,
 'solving time': 10.669,
 'annealing time': 0.0,
 'simulation time': 0.0,
 'quantum success number': 0,
 'quantum conflict number': 0,
 'quantum one time solve number': 0,
 'is satisfiable': True,
 'is sat': True}

In [5]:
all_result = {}
dir_name = 'cnf_examples/test'
for filename in os.listdir(dir_name):
    janusq_res = solve_by_janusct(os.path.join(dir_name, filename), verb=False)
    minisat_res = solve_by_minisat(os.path.join(dir_name, filename), verb=False)
    for key in janusq_res:
        if key not in all_result:
            all_result[key] = {
                'janus': 0,
                'minisat': 0
            }
        all_result[key]['janus'] += janusq_res[key]
        all_result[key]['minisat'] += minisat_res[key]

print(all_result)    

{'restarts': {'janus': 12, 'minisat': 19}, 'conflicts': {'janus': 143, 'minisat': 1383}, 'conflict cost': {'janus': 1.4689999999999996, 'minisat': 12.712999999999997}, 'decisions': {'janus': 34, 'minisat': 1814}, 'decisions cost': {'janus': 0.4750000000000001, 'minisat': 2.4249999999999994}, 'propagations': {'janus': 4431, 'minisat': 32252}, 'propagations cost': {'janus': 5.236, 'minisat': 23.714}, 'conflict literals': {'janus': 847, 'minisat': 8726}, 'actual CPU time': {'janus': 37.84356, 'minisat': 1.107393}, 'solving time': {'janus': 18.371, 'minisat': 47.944}, 'annealing time': {'janus': 7.340000000000001, 'minisat': 0.0}, 'simulation time': {'janus': 36.08566, 'minisat': 0.0}, 'quantum success number': {'janus': 140, 'minisat': 0}, 'quantum conflict number': {'janus': 227, 'minisat': 0}, 'quantum one time solve number': {'janus': 0, 'minisat': 0}, 'is satisfiable': {'janus': 12, 'minisat': 12}, 'is sat': {'janus': 12, 'minisat': 12}}


In [6]:
import pandas as pd

df = pd.DataFrame(all_result)
df

,restarts,conflicts,conflict cost,decisions,decisions cost,propagations,propagations cost,conflict literals,actual CPU time,solving time,annealing time,simulation time,quantum success number,quantum conflict number,quantum one time solve number,is satisfiable,is sat
janus,12,143,1.469,34,0.475,4431,5.236,847,37.843560,18.371,7.34,36.08566,140,227,0,12,12
minisat,19,1383,12.713,1814,2.425,32252,23.714,8726,1.107393,47.944,0.00,0.00000,0,0,0,12,12
